In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
# libary to read audio in torch
import torchaudio
from torch.nn import CTCLoss
from jiwer import wer
import soundfile as sf

softmax = torch.nn.LogSoftmax(dim=1)
ctcloss = CTCLoss()

Importing model & dataset

In [24]:
# load model and processor
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

In [25]:
# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# example of tokenizing an audio file
input_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest", sampling_rate = ds[0]["audio"]["sampling_rate"]).input_values  # Batch size 1

In [26]:
#select an audio file and process it
#question: is it possible to process the audio after adding noise to it and backpropagate the loss?
audio = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest", sampling_rate = ds[0]["audio"]["sampling_rate"]).input_values
sampling_rate = ds[0]["audio"]["sampling_rate"]

#sentence we want our model to predict
target = "THE CAT IS INSIDE MY BAG AND IT ROLLS ON THE FLOOR"
#assuming: target is a list which contains one sentence
target = [c for c in target]
# convert to tensor logits using the tokenizer
target_logits = processor.tokenizer.convert_tokens_to_ids(target)
target_logits = torch.tensor(target_logits)

#load everything to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
audio = audio.to(device)
target_logits = target_logits.to(device)

In [27]:
def ctc_loss(logits, targets):
    #this function only tested for batch_size = 1
    input_lengths = torch.tensor([logits.shape[0]])
    target_lengths = torch.tensor([targets.shape[0]])
    return ctcloss(logits, targets, input_lengths, target_lengths)

In [31]:
def loss_function(audio, noise, target_logits, model):
    audio_perturbed = audio + noise
    #compute dB_x
    dB_x = 
    #compute dB_delta
    ...
    #compute dB_x_delta
    ...
    #compute logits
    logits = model(audio_perturbed).logits
    logits = softmax(logits[0])
    logits = logits.unsqueeze(1)
    #compute ctc loss
    loss = ctc_loss(logits, target_logits)
    #compute total loss
    ...
    loss = loss

    return loss


In [32]:
noise = torch.zeros_like(audio)
noise.requires_grad = True

lr = 1e-1
optimizer = torch.optim.Adam([noise], lr=lr)

losses = []
min_loss = 100000
min_noise = None
for i in range(1000):
    optimizer.zero_grad()
    loss = loss_function(audio, noise, target_logits, model)
    loss.backward()
    optimizer.step()
    itemized_loss = loss.item()
    losses.append(itemized_loss)
    if itemized_loss < min_loss:
        min_loss = itemized_loss
        min_noise = noise.detach().cpu().numpy()
    if i % 10 == 0:
        print(losses[-10:])

[27.828081130981445]
[24.386919021606445, 13.813969612121582, 11.216731071472168, 9.805255889892578, 9.317750930786133, 8.759771347045898, 9.356657981872559, 8.448748588562012, 8.402827262878418, 8.880524635314941]
[8.306851387023926, 7.964898586273193, 7.836520195007324, 7.748620986938477, 7.669915676116943, 7.517343521118164, 7.5303778648376465, 7.535662651062012, 7.7495551109313965, 7.684903144836426]
[7.48953104019165, 7.380371570587158, 7.442655086517334, 7.3777594566345215, 7.167582988739014, 7.004438400268555, 6.784716606140137, 6.724461555480957, 6.5612335205078125, 6.5858235359191895]
[6.571091175079346, 6.461272716522217, 6.3936028480529785, 6.291732311248779, 6.27535343170166, 6.24393367767334, 6.211443901062012, 6.226598739624023, 6.445048809051514, 6.444520473480225]
[6.476072311401367, 6.417241096496582, 6.378167152404785, 6.2178144454956055, 6.142199516296387, 6.031375885009766, 5.951697826385498, 5.912008285522461, 5.873651027679443, 5.761438369750977]
[5.68957948684692

KeyboardInterrupt: 

In [33]:
#test = audio + torch.tensor(min_noise).to(device)
test = audio + noise
logits = model(test).logits
#print predicted sentence
print(processor.batch_decode(torch.argmax(logits, dim=-1)))

['THE CAT<unk>IS<unk>INSIDE<unk>MY<unk>BAG<unk>AND<unk>IT<unk>ROLLS</s>ON<unk>THE<unk>FLOOR']


In [12]:
torchaudio.save("test.wav", test.detach().cpu(), sampling_rate)

### EVERYTHING BELOW IS TRASH DO NOT BOTHER WITH IT

unless you need inspiration or something

In [34]:
target_sentence = ds[0]["text"]
target_sentence = [c for c in target_sentence]
target_logits = processor.tokenizer.convert_tokens_to_ids(target_sentence)
target_logits = torch.tensor(target_logits)

test_target_sentence = ds[5]['text']
test_target_sentence = [c for c in test_target_sentence]
test_target_logits = processor.tokenizer.convert_tokens_to_ids(test_target_sentence)
test_target_logits = torch.tensor(test_target_logits)

In [42]:
softmaxed_logits = softmax(output_logits)

# show predicted sentence
predicted_sentence = processor.decode(torch.argmax(softmaxed_logits, dim=-1))
print(predicted_sentence)
print(ds[0]['text'])

MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL
MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL


In [66]:
ctc_loss(output_logits.unsqueeze(1), target_logits.unsqueeze(0), torch.tensor([output_logits.shape[0]]), torch.tensor([target_logits.shape[0]]))

tensor(-44.7382, grad_fn=<MeanBackward0>)

In [62]:
torch.tensor([[target_logits.shape[0]]])

tensor([[89]])

In [13]:
output_logits = model(audio).logits
output_logits = softmax(output_logits[0])

# get predicted sentence
predicted_sentence = processor.decode(torch.argmax(output_logits, dim=-1))
print(predicted_sentence)

MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL


In [29]:
# test_target_sentence = ds[1]['text']
test_target_sentence = ds[2]['text']
test_target_sentence = [c for c in test_target_sentence]
test_target_logits = processor.tokenizer.convert_tokens_to_ids(test_target_sentence)
test_target_logits = torch.tensor(test_target_logits)

ctc_loss(output_logits, test_target_logits.unsqueeze(0))

tensor(1263.2513, device='cuda:0', grad_fn=<MeanBackward0>)

In [138]:
test_target_logits.shape

torch.Size([90])

In [11]:
# tokenize
input_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest", sampling_rate = ds[0]["audio"]['sampling_rate']).input_values  # Batch size 1

# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

In [5]:
audio = torch.Tensor(ds[0]["audio"]["array"]).unsqueeze(0)
rate = ds[0]["audio"]["sampling_rate"]
torchaudio.save("test.wav", audio, rate)

In [12]:
input_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest").input_values  # Batch size 1
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
transcription

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


['MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL']

In [66]:
good_transcription = 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'

transcription = "THE CAT JUMPED OVER THE FOX WHERE IT HAS SHOWN US THE WORLD"

In [67]:
target = processor.tokenizer.convert_tokens_to_ids([c for c in transcription[0]])
target = torch.Tensor(target).unsqueeze(0).long()
target_shape = target[0].shape[0]
logits_shape = logits[0].shape[0]

In [68]:
ctcloss(log_probs=softmax(logits[0]), targets=target[0], input_lengths=[logits_shape], target_lengths=[target_shape])

tensor(-170.5979, grad_fn=<MeanBackward0>)

In [ ]:
predicted_ids

In [ ]:
predicted_characters = processor.tokenizer.convert_ids_to_tokens(predicted_ids[0].tolist())
predicted_characters

In [ ]:
transcription_list = [c for c in transcription[0]]
transcription_list

In [ ]:
processor.tokenizer.convert_tokens_to_ids(transcription_list)

In [ ]:
import itertools
# remove consecutive duplicates
result = [k for k, g in itertools.groupby(predicted_ids[0])]
# remove blanks
result = [x for x in result if x != 0]
#count
len(result)

In [ ]:
(predicted_ids>0).sum()

In [ ]:
# convert transcription to logits
transcription_logits = processor(transcription, return_tensors="pt", padding="longest").input_values

In [ ]:
# load files extra2a, extra2b and infer
files = ["extra2a.wav", "extra2b.wav"]
# read audios
audio, rate = torchaudio.load(files[0])
audio2, rate2 = torchaudio.load(files[1])

In [76]:
#infer audio1
input_values = processor(audio[0], return_tensors="pt", padding="longest", sampling_rate=rate).input_values  # Batch size 1

In [79]:
audio.shape

torch.Size([1, 93680])

In [80]:
input_values.shape

torch.Size([1, 93680])

In [ ]:
# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.batch_decode(predicted_ids)
transcription

In [5]:
#infer audio1
file = "extra_0a.wav"
audio, rate = torchaudio.load(file)
audio = audio
input_values = processor(audio[0], return_tensors="pt", padding="longest", sampling_rate=rate).input_values  # Batch size 1
# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.batch_decode(predicted_ids)
transcription

['THAT DAY THE MERCHANT GAVE THE BOY PERMISSION TO BUILD THE DIS']

In [6]:
logits[0].shape

torch.Size([257, 32])

Goal: Minimize 

$ dB_x(\delta) + c l(x+\delta, t) $

where 

$dB_x(\delta)$ is the strength of the noise compared to the signal

$c$: tradeoff parameter between being adversarial and being close to the original signal

$l(x+\delta, t)$ : the loss between the (disturbed signal prediction?) and the target sentence to become adversarial t?

we define $l$ as the CTC-loss, so we can say:

$-log Pr(t | x+\delta) $

In [24]:
target = ["THE CAT IS INSIDE MY BAG AND IT ROLLS ON THE FLOOR"]
#assuming: target is a list which contains one sentence
target = [c for c in target[0]]
# convert to tensor logits
target_logits = processor.tokenizer.convert_tokens_to_ids(target)
target_logits = torch.tensor(target_logits)

#compute adversarial example
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
audio = audio.to(device)
audio = processor(audio, return_tensors="pt", padding="longest", sampling_rate=rate).input_values
audio = audio[0].to(device)
target_logits = target_logits.to(device)

In [25]:
# minimize dB_x(delta) - logPr(t|f(x+delta)), such that dB_x(delta) < eps
# delta = argmin dB_x(delta) - logPr(t|f(x+delta))
# dB_x(delta) = 20*log10(||x+delta||_2 / ||delta||_2)
# x = audio
# t = target transcription
# f = model
# eps = max distortion
# delta = perturbation


# define loss function
def loss_function(audio, noise, target_logits, model, eps, ctc_constant):
    audio_perturbed = audio + noise
    # print(input_values.shape)
    # print(audio_perturbed.shape)
    #audio: clean audio
    # calculate dB_x, dB_delta, dB_x(delta) , where delta is perturbed_noise - clean_audio
    dB_x = 20*torch.log10(torch.norm(audio))
    # calculate dB_delta
    # add 1e-10 to avoid log of zero
    dB_delta = 20*torch.log10(torch.norm(noise+1e-10))
    # calculate dB_x(delta)
    dB_x_delta = dB_delta - dB_x

    # calculate logPr(t|f(x+delta))
    logits = model(audio_perturbed).logits
    logits = logits[0] # remove batch dimension
    logits = softmax(logits)
    # print(target_logits)
    # print(target_logits.shape, target_logits.dtype)
    # print(logits)
    # print(logits.shape, logits.dtype)

    # print(logits.shape[0])
    # print(target_logits.shape[0])
    # print(target_logits)
    # print(logits)
    logPr = ctcloss(logits, target_logits, [logits.shape[0]], [target_logits.shape[0]])
    # calculate loss
    # print("dB_x_delta, logPr")
    # print(dB_x_delta, logPr)
    # loss = dB_x_delta - ctc_constant * logPr
    loss = - logPr

    # check if dbloss is smaller than eps
    if dB_x_delta < eps:
        return loss
    else:
        # print(dB_x_delta, eps)
        # return None
        return loss

In [26]:
# define eps
eps = 10
# define number of iterations
n = 5000
# define learning rate
lr = 1e-1
# define perturbed audio: start with clean audio
noise = torch.zeros_like(audio).requires_grad_(True)
# define optimizer
optimizer = torch.optim.Adam([noise], lr=lr)
ctc_constant = 1

# loop over n iterations
for i in range(n):
    # set gradients to zero
    optimizer.zero_grad()
    # calculate loss
    loss = loss_function(audio, noise, target_logits, model, eps, ctc_constant=ctc_constant)
    # break if loss is None
    if loss is None:
        break
    print("final loss")
    print(loss)      
    # calculate gradients
    loss.backward()
    # update perturbation
    optimizer.step()
#    print(audio_pert)
# save adversarial example
audio_pert = (audio+noise).detach().to("cpu")
torchaudio.save("adversarial_one.wav", audio_pert, rate)

final loss
tensor(5.8390, device='cuda:0', grad_fn=<NegBackward0>)
final loss
tensor(5.9684, device='cuda:0', grad_fn=<NegBackward0>)


KeyboardInterrupt: 

In [149]:
loss

tensor(22.0689, device='cuda:0', grad_fn=<SubBackward0>)

In [52]:
audio.dtype

torch.float32

In [50]:
audio_pert

array([[-0.3960265 ,  0.37243792, -0.37965736, ...,  0.33345932,
        -0.42485094,  0.32958943]], dtype=float32)

In [32]:
# display audio object
audio_pert

tensor([[-0.3785, -0.3786, -0.3785,  ..., -0.4242, -0.4231, -0.4313]],
       requires_grad=True)

In [33]:
audio

tensor([[ 0.0003,  0.0002,  0.0002,  ..., -0.0454, -0.0443, -0.0526]])

In [ ]:
softmaxed = torch.nn.Softmax(dim=1)
probs = softmaxed(model(perturbed_audio).logits)

In [ ]:
probs.sum()

In [ ]:
target_sentence = "THE WILL BURN YOU TO A CRISP"
#convert to tokens

In [ ]:
perturbed_audio = audio + audio_pert
dB_x = 20 * torch.log10(torch.norm(audio) / torch.norm(audio_pert))
# calculate logPr(t|f(x+delta))
logits = model(perturbed_audio).logits
pred = processor.batch_decode(torch.argmax(logits, dim=-1))

#print(target)
print(pred)

In [ ]:
type(pred)